## Imports and notebook configuration

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython import display
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# %matplotlib inline
%matplotlib ipympl

### General imports
import os
import sys
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from datetime import datetime
from joblib import Parallel, delayed
from multiprocessing import Manager, Lock
import healpy as hp
import pickle

from scipy.fft import fft, ifft, fftfreq
import matplotlib.colors as clrs

plt.rc('figure',figsize=(10,6))
plt.rc('font',size=12)

### Astropy configuration
from astropy.visualization import astropy_mpl_style, quantity_support
quantity_support()
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_sun



#### QUBIC IMPORT
from qubicpack.utilities import Qubic_DataDir
from qubicpack.qubicfp import qubicfp
from qubicpack.pix2tes import pix2tes, tes2pix

from qubic.lib.Calibration import Qfiber
from qubic.lib.Calibration import Qselfcal
from qubic.lib.Qutilities import progress_bar
# from qubic.lib.QdataHandling import display_healpix_map, identify_scans
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument
import qubic.lib.Calibration.Qfiber as ft




def iQS2iQP(indexQS):
    qpnumi, qpasici = qp.pix2tes.pix2tes(indexQS+1)
    return qpnumi+(qpasici-1)*128-1

def iQP2iQS(indexQP):
    QStesnum = qp.pix2tes.tes2pix(indexQP%128+1, indexQP//128+1)
    return QStesnum-1

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)

plt.rc('figure',figsize=(20,20))
plt.rc('font',size=12)


### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = ["/Users/huchet/qubic/qubic/scripts/MoonProject/", "/Users/huchet/Documents/code/scripts/", "/Users/huchet/Documents/code/data/"] #[os.environ['QUBIC_DATADIR']+'scripts/MoonProject/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
# import fitting as fit
# import time_domain_tools as tdt
# import useful_functions as uf
import qubic.scripts.MoonProject.pipeline_moon_plotting as pmp
import qubic.scripts.MoonProject.pipeline_moon_functions as pmf
### Import functions that are not in lib anymore
from some_functions import find_file, identify_scans, display_healpix_map
# from scripts.MoonProject.some_functions_old import find_file, identify_scans, display_healpix_map




In [ ]:
# Un peu plus conservateur mais il se peut que certains soient encore à retirer
visibly_ok_arr = [False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False,  False, False, False, False,  True,  True, False, # TES 18 set to False because flux jumps
 False,  True,  True, False, False, False, False, False,  True,  True, False, False,
 False,  True, False,  True, False, False, False, False, False,  True, False, False,
  True,  True,  True,  True, False,  True,  True, False,  True, False,  True, False,
  True,  True,  True,  True, False, False,  True, False,  True,  True, False,  True,
  True, False, False,  True, False, False,  True,  True,  True,  True,  True,  True,
 False,  True,  True, False,  True, False, False, False, False,  True,  True,  True,
 False, False,  True, False, False, False, False, False, False, False,  True, False,
 False, False, False,  True,  True, False, False, False, False,  True, False, False,
 False, False, False, False, False, False, False, False,  True,  True,  True, False,
  True, False,  True,  True,  True,  True,  True, False, False, False, False, False,
 False, False, False, False, False,  True, False,  True, False, False, False, False,
 False,  True,  True, False, False,  True,  True, False, False, False, False, False,
 False,  True,  True,  True,  True, False,  True,  True,  True,  True, False,  True,
  True,  True, False, False,  True,  True,  True,  True,  True, False, False, False,
  True,  True,  True, False,  True,  True,  True,  True, False, False,  True,  True,
  True,  True,  True, False,  True,  True,  True,  True,  True,  True,  True,  True,
 False,  True, False, False,  True, False,  True, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
  True,  True,  True,  True, False, False,  True,  True, False,  True, False,  True,
 False, False, False, False]

In [ ]:
visibly_ok_arr[55 - 1] = False

In [ ]:
# These are the visual inspection results
visibly_ok_arr = np.array([False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
 False,  True,  True, False, False, False, False, False,  True, False, False, False,
 False, False, False,  True, False, False, False, False, False,  True, False, False,
  True, False,  True,  True, False, False, False, False,  True, False,  True, False,
  True,  True,  True,  True, False, False,  True, False,  True, False, False, False,
  True, False, False,  True, False, False, False, False,  True, False,  True, False,
 False,  True,  True, False, False, False, False, False, False, False,  True,  True,
 False, False,  True, False, False, False, False, False, False, False,  True, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False,  True,  True,  True, False,
  True, False, False,  True,  True,  True,  True, False, False, False, False, False,
 False, False, False, False, False,  True, False,  True, False, False, False, False,
 False,  True,  True, False, False, False, False, False, False, False, False, False,
 False,  True,  True, False, False, False, False,  True,  True,  True, False, False,
  True,  True, False, False, False, False, False,  True,  True, False, False, False,
  True,  True,  True, False, False, False,  True, False, False, False, False, False,
  True,  True,  True, False,  True,  True,  True,  True, False,  True,  True, False,
 False,  True, False, False,  True, False,  True, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
  True,  True,  True,  True, False, False,  True,  True, False,  True, False,  True,
 False, False, False, False,])

# These are visibly OK but are actually ghosts
last_ghosts = [177, 214, 238, 241, 243, 244, 250]
for ghost in last_ghosts:
    visibly_ok_arr[ghost - 1] = False

# This is a bad fit too hard to fix
bad_fits = [247]
for bad_fit in bad_fits:
    visibly_ok_arr[bad_fit - 1] = False

# Stack the maps at the Moon position (test)

In [ ]:
# allTESNum, allmaps = pickle.load( open( "NEW2025_v2-allmaps-Jan14-2022.pkl", "rb" ) )
# allTESNum, allmaps = pickle.load( open( "202506-allmaps-Jan14-2022_unturned.pkl", "rb" ) )
# allTESNum, allmaps = pickle.load( open( "202506-allmaps-Jan14-2022_unturned_RADec.pkl", "rb" ) )
# allTESNum, allmaps = pickle.load( open( "202506-allmaps-Jan14-2022_unturned_azelv2.pkl", "rb" ) )
allTESNum, allmaps = pickle.load( open( "202506-allmaps-Jan14-2022_unturned_azelv3.pkl", "rb" ) )
# print(allTESNum)
# print(np.shape(allmaps))

reso = 5
xs = 201
# myrotinit = [-1, -4, 0]
myrotinit = [0, 0, 0]
allamp = np.zeros(len(allTESNum))
allerramp = np.zeros(len(allTESNum))
allFWHM = np.zeros(len(allTESNum)) * np.nan
allerrFWHM = np.zeros(len(allTESNum))
allxy = np.zeros((len(allTESNum), 2)) * np.nan
allerrxy = np.zeros((len(allTESNum), 2))

moon_fit = []

for i in range(len(allTESNum)):
    # idx = np.where(np.array(allTESNum-1)==i)[0][0]
    if not visibly_ok_arr[i]:
        moon_fit.append([(np.nan, np.nan), np.nan, np.nan])
        continue
    m = pmf.fit_one_tes(allmaps[i, :], xs, reso, rot=myrotinit, verbose=False, doplot=False)
    allFWHM[i] = m.values[3] * pmf.conv_reso_fwhm
    allerrFWHM[i] = m.errors[3] * pmf.conv_reso_fwhm
    allamp[i] = m.values[0] * pmf.conv_reso_fwhm
    allerramp[i] = m.errors[0] * pmf.conv_reso_fwhm
    # allxy[i, :] = m.values[1:3]
    # allerrxy[i, :] = m.errors[1:3]
    allxy[i, :] = [m.values[2], m.values[1]] # m.values contains amplitude, elevation, azimuth, sigma
    allerrxy[i, :] = [m.errors[2], m.errors[1]]
    print('TES#{0}: FWHM = {1:5.2f}'.format(i + 1, m.values[3] * pmf.conv_reso_fwhm))
    moon_fit.append([allxy[i, :], allFWHM[i], allamp[i]])

In [ ]:
%%script echo skipping

# We then perform it for all TES and store the results

# print(allTESNum)
# print(np.shape(allmaps))

reso = 5 #4
xs = 201
allamp = np.zeros(len(allTESNum))
allerramp = np.zeros(len(allTESNum))
allFWHM = np.zeros(len(allTESNum)) * np.nan
allerrFWHM = np.zeros(len(allTESNum))
allxy = np.zeros((len(allTESNum), 2)) * np.nan
allerrxy = np.zeros((len(allTESNum), 2))

moon_fit = []

for i in range(len(allTESNum)):
    # idx = np.where(np.array(allTESNum-1)==i)[0][0]
    if not visibly_ok_arr[i]:
        moon_fit.append([(np.nan, np.nan), np.nan, np.nan])
        continue
    m = pmf.fit_one_tes(allmaps[i,:], xs, reso, rot=myrotinit, verbose=False, renorm=True, doplot=False)
    allFWHM[i] = m.values[3] * pmf.conv_reso_fwhm
    allerrFWHM[i] = m.errors[3] * pmf.conv_reso_fwhm
    allamp[i] = m.values[0] * pmf.conv_reso_fwhm
    allerramp[i] = m.errors[0] * pmf.conv_reso_fwhm
    # allxy[i, :] = m.values[1:3]
    # allerrxy[i, :] = m.errors[1:3]
    allxy[i, :] = [m.values[2], m.values[1]] # m.values contains amplitude, elevation, azimuth, sigma
    allerrxy[i, :] = [m.errors[2], m.errors[1]]
    print('TES#{0}: FWHM = {1:5.2f}'.format(i + 1, m.values[3] * pmf.conv_reso_fwhm))
    moon_fit.append([allxy[i, :], allFWHM[i], allamp[i]])

In [ ]:
# tot_map = np.zeros_like(allmaps[0, :])
# nside = 340
nside = int(np.sqrt(len(allmaps[0])/12))
print("NSIDE = {}".format(nside))
xs = 401
# tot_map = np.zeros(12*nside**2) + hp.UNSEEN

mm = np.zeros(12*nside**2)
tot_map = hp.gnomview(mm, reso=4, return_projected_map=True, xsize=xs, no_plot=True).data
print(np.shape(tot_map))

# i = 238 - 1
# moon_pos = moon_fit[allTESNum[i] - 1][0]
# center_map = (moon_pos[0], -moon_pos[1])
# hp.gnomview(allmaps[i], reso=4, rot=center_map, min=-5e3, max=1.2e4, return_projected_map=True, xsize=xs, no_plot=False)
# sys.exit()
maps_stack = np.zeros((xs, xs, len(allTESNum)))
# print(np.shape(allmaps))
for i in range(len(allTESNum)):
    # print(i)
    if not visibly_ok_arr[i]:
        continue
    ok = allmaps[i, :] != hp.UNSEEN
    mm = np.zeros(12*nside**2)
    mm[ok] = allmaps[i, ok]
    # mm[~ok] = hp.UNSEEN
    # hp.gnomview(mm, reso=10, min=-5e3, max=1.2e4, 
    #             title=allTESNum[i], rot=center)
    # tot_map += mm
    # print(moon_fit[allTESNum[i] - 1][0])
    # moon_pos = moon_fit[allTESNum[i] - 1][0]
    moon_pos = [0, 0]
    center_map = (moon_pos[0], moon_pos[1])
    maps_stack[:, :, i] = hp.gnomview(mm, reso=5, rot=center_map, min=-5e3, max=1.2e4, return_projected_map=True, xsize=xs, no_plot=True).data
    # tot_map += mapxy

# Alledgedly faster than masked arrays
# https://stackoverflow.com/questions/22049140/how-can-i-ignore-zeros-when-i-take-the-median-on-columns-of-an-array
tot_map = np.apply_along_axis(lambda v: np.median(v[np.nonzero(v)]), 2, maps_stack)
tot_map[np.isnan(tot_map)] = 0
# tot_map = np.median()
print(np.shape(tot_map))

In [ ]:
# tot_map_plot = tot_map.copy()
# vmin = 1
# vmax = 4e4
# tot_map_plot[tot_map_plot < vmin] = vmin
# norm = clrs.LogNorm(vmin=vmin, vmax=vmax)
# plt.figure()
# plt.imshow(tot_map_plot, norm=norm)
# plt.show()

plt.figure()
plt.imshow(tot_map, vmin=-5e3, vmax=1.2e4, origin="lower")
plt.show()

In [ ]:
%%script echo skipping
# Save the stacked map for next step
# pickle.dump( tot_map, open( "stack_NEW2025_v2-allmaps-Jan14-2022.pkl", "wb" ) )
# pickle.dump( tot_map, open( "stack_202506-allmaps-Jan14-2022_unturned.pkl", "wb" ) )
pickle.dump( tot_map, open( "stack_202506-allmaps-Jan14-2022_unturned_RADec.pkl", "wb" ) )

In [ ]:
%%script echo skipping

tot_map_mean = np.apply_along_axis(lambda v: np.mean(v[np.nonzero(v)]), 2, maps_stack)
tot_map_mean[np.isnan(tot_map_mean)] = 0

plt.figure()
plt.imshow(tot_map_mean)
plt.show()

In [ ]:
%%script echo skipping

def sig_clip_mean(stack_centered):
    sig_clip = stack_centered[stack_centered < 3 * np.std(stack_centered)]
    return np.mean(sig_clip)

tot_map_mean_v2 = np.apply_along_axis(sig_clip_mean, 2, (maps_stack - np.full((xs, xs, len(allTESNum)), np.expand_dims(tot_map_mean, axis=2))))
tot_map_mean_v2[np.isnan(tot_map_mean_v2)] = 0
tot_map_mean_v2 += tot_map_mean

plt.figure()
plt.imshow(tot_map_mean_v2)
plt.show()